In [ ]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
# from quantuminspire.projectq.backend_qx import QIBackend

import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
print('Enter mail address')
email = input()

print('Enter password')
password = getpass()

server_url = r'https://api.quantum-inspire.com'
authentication = BasicAuthentication(email, password)
qi = QuantumInspireAPI(server_url, authentication)

In [ ]:
#https://stackoverflow.com/questions/9600801/evenly-distributing-n-points-on-a-sphere
#Create n evenly distributed points on a sphere

num_pts = 1000
indices = np.arange(0, num_pts, dtype=float) + 0.5

theta = np.arccos(1 - 2*indices/num_pts)
phi = (np.pi * (1 + 5**0.5) * indices)%(2*np.pi)
coords=[]

for theta_elem,phi_elem in zip(theta,phi):
    coords.append((theta_elem,phi_elem))

x, y, z = np.cos(phi) * np.sin(theta), np.sin(phi) * np.sin(theta), np.cos(theta);

# Plot the surface.
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, template="plotly_white")

fig.show()

In [ ]:
def parameterized_QACM(loc):
    θ,ϕ=loc
    qasm = '''
    version 1.0

    qubits 5

    # initialize the state
    prep_z q[2]
    Ry q[0], {0}
    Rz q[0], {1}
    
    #preparation
    Ry q[2], 1.107149
    #rewrite CNOT q[0],q[4] and CNOT q[4],q[0] usign nearest neighbors
    CNOT q[2],q[1]

    Ry q[1], 0.729728
    CNOT q[1],q[2]

    Ry q[2], 0.463648
    SWAP q[0],q[2]

    #copying
    CNOT q[2], q[0]
    CNOT q[2], q[1]
    CNOT q[0], q[2]
    CNOT q[1], q[2]

    #Rotate back and measure
    Rz q[2], {2}
    Ry q[2], {3}
    Rz q[0], {2}
    Ry q[0], {3}
    Measure_z q[2]
    Measure_z q[0]
    '''
    return qasm.format(θ,ϕ,-ϕ,-θ)

In [ ]:
N_shots=16384
target_points=np.array(coords)
# save the data, optional
np.savetxt("target_points_starmon.csv",target_points)

In [ ]:
# iterate over the sphere, run task
backend_type = qi.get_backend_type_by_name('Starmon-5')

copied_data=[]
index=0
for points in target_points:
    qasm=parameterized_QACM(points)
    result = qi.execute_qasm(qasm, backend_type=backend_type, number_of_shots=N_shots)
    hist=result.get('histogram', {})

    #test both the copies
    marg_prob0_qubit0=hist["0"]+hist["4"] #calculate the marginal distribution, p=p(00000)+p(00100)
    marg_prob0_qubit2=hist["0"]+hist["1"] #calculate the marginal distribution, p=p(00000)+p(00001)
    marg_prob1_qubit0=1-marg_prob0_qubit0
    marg_prob1_qubit2=1-marg_prob0_qubit2

    copied_data.append([marg_prob0_qubit0,marg_prob1_qubit0, marg_prob0_qubit2, marg_prob1_qubit2])
    index=index+1
    print(str(index)+"/"+str(num_pts))
    print("Execution time: ", result.get('execution_time_in_seconds',{}))
    print(points)
    print("Qubit 0: ", [marg_prob0_qubit0,marg_prob1_qubit0])
    print("Qubit 2: ", [marg_prob0_qubit2,marg_prob1_qubit2])
    with open('results_starmon.txt', 'a') as file:
        file.write(str(points[0])+"\t"+str(points[1])+"\t"+str(marg_prob0_qubit0)+"\t"+str(marg_prob1_qubit0)+"\t"+
                   str(marg_prob0_qubit2)+"\t"+str(marg_prob1_qubit2)+"\n")
    

copied_data=np.array(copied_data)

In [13]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,copied_data)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_2","prob_1_qubit_2"])

In [14]:
data_sheet.head()

,θ,ϕ,prob_0_qubit_0,prob_1_qubit_0,prob_0_qubit_2,prob_1_qubit_2
0,0.044725,5.083204,0.723816,0.276184,0.734680,0.265320
1,0.077479,2.683240,0.813232,0.186768,0.804138,0.195862
2,0.100042,0.283277,0.804932,0.195068,0.804688,0.195312
3,0.118391,4.166499,0.815308,0.184692,0.811462,0.188538
4,0.134265,1.766536,0.817810,0.182190,0.798340,0.201660


In [15]:
# save to excel, optional
data_sheet.to_excel("data_sheet_starmon.xlsx")

In [16]:
data_sheet=pd.read_excel("data_sheet_starmon.xlsx")

In [17]:
#FIDELITY OF FIRST COPY
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_2/(5/6), template="plotly_white")

fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_2))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_2))

Number of points:  1000
Average fidelity:  0.7334130249023437
Standard deviation:  0.04560676507022329


In [18]:
#FIDELITY OF SECOND COPY
# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0/(5/6), template="plotly_white")

fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

Number of points:  1000
Average fidelity:  0.73401171875
Standard deviation:  0.02931495615240185


In [19]:
#CHECK SYMMETRY, I.E. PLOT fidelity of qubit 0/fidelity of qubit 2

df = px.data.iris()
ratio=data_sheet.prob_0_qubit_2/data_sheet.prob_0_qubit_0
fig = px.scatter_3d(df, x, y, z, color=ratio, template="simple_white")

fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(ratio))
print("Standard deviation: ", np.std(ratio))

Number of points:  1000
Average fidelity:  0.9989801847072085
Standard deviation:  0.044077926393166544
